You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [8]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [9]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [ ]:
import os
print(os.environ["OPENAI_API_KEY"])

In [3]:
from dotenv import load_dotenv
_ = load_dotenv()

In [4]:
import os


api_key = os.getenv("API_KEY")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [15]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [16]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [17]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [18]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [19]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 
Title: The Future of Artificial Intelligence: Latest Trends and Key Players

Introduction:
- Brief overview of Artificial Intelligence and its impact on various industries
- Importance of staying updated on the latest trends in AI

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning advancements
- Natural language processing innovations
- AI ethics and regulation developments

2. Key Players in the AI Industry:
- Google
- Microsoft
- IBM
- Amazon

3. Noteworthy News in A

- Display the results of your execution as markdown in the notebook.

In [20]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends and Key Players

## Introduction

Artificial Intelligence (AI) has revolutionized various industries, from healthcare to finance, with its ability to analyze data, automate processes, and make predictions. Staying updated on the latest trends in AI is crucial for tech enthusiasts, business professionals, students, and researchers to understand the evolving landscape of this transformative technology.

## Latest Trends in Artificial Intelligence

### Machine Learning Advancements
Machine learning, a subset of AI, has seen significant advancements in recent years. Algorithms are becoming more sophisticated, enabling machines to learn from data and improve their performance without explicit programming. This has led to more efficient data analysis, pattern recognition, and predictive modeling in various applications, from recommendation systems to autonomous vehicles.

### Natural Language Processing Innovations
Natural Language Processing (NLP) has made tremendous progress, allowing machines to understand and generate human language. Voice assistants and chatbots are examples of NLP applications that are reshaping how we interact with technology. These advancements in NLP have not only improved user experience but also opened up new possibilities for communication and information retrieval in diverse domains.

### AI Ethics and Regulation Developments
As AI technologies become more pervasive, concerns about ethics and regulation have come to the forefront. Debates on bias in algorithms, data privacy, and the ethical use of AI are shaping the future of AI development. It is crucial for organizations and policymakers to address these ethical considerations and establish guidelines to ensure the responsible and beneficial deployment of AI technologies.

## Key Players in the AI Industry

### Google
Google is at the forefront of AI research and development, with projects ranging from self-driving cars to healthcare diagnostics. Its AI-powered products like Google Assistant and Google Translate showcase the company's commitment to AI innovation. Google's contributions to AI ethics and accessibility have also influenced industry standards and best practices.

### Microsoft
Microsoft has invested heavily in AI, integrating it into its products and services. Azure AI platform, Cortana, and Microsoft Cognitive Services are some of the offerings that demonstrate Microsoft's expertise in AI technologies. The company's focus on democratizing AI and making it accessible to developers and businesses has contributed to the widespread adoption of AI solutions.

### IBM
IBM's Watson platform is a pioneer in AI applications, offering solutions for data analysis, customer service, and healthcare. The company's research in AI ethics and responsible AI practices sets it apart in the industry. IBM's collaborative approach to AI development, through partnerships and open-source initiatives, has fostered innovation and knowledge sharing in the AI community.

### Amazon
Amazon's AI initiatives focus on improving customer experience through personalized recommendations, supply chain optimization, and voice-enabled devices like Alexa. The company's investments in AI research continue to drive innovation across its business units. Amazon's customer-centric approach to AI development and its emphasis on user personalization have positioned the company as a leader in AI-driven customer engagement.

## Noteworthy News in Artificial Intelligence

### Recent Breakthroughs in AI Research
Researchers are constantly pushing the boundaries of AI, achieving milestones in areas like reinforcement learning, computer vision, and robotics. These breakthroughs have the potential to revolutionize various industries and improve our daily lives. From autonomous systems to intelligent assistants, AI technologies are creating new opportunities for innovation and problem-solving.

### AI Applications in Healthcare, Finance, and Other Sectors
AI is transforming healthcare with predictive analytics, personalized medicine, and medical imaging technologies. In finance, AI-powered tools are enhancing decision-making, risk management, and fraud detection. The applications of AI are diverse and impactful across multiple sectors, driving efficiency, accuracy, and insights that were previously unattainable.

### Updates on AI-Powered Technologies
Self-driving cars, virtual assistants, and smart home devices are examples of AI-powered technologies that are becoming more prevalent in our society. These innovations are reshaping how we work, communicate, and live, offering convenience and efficiency. As AI continues to evolve, its integration into everyday devices and systems is creating a more connected and intelligent environment for individuals and businesses.

## Conclusion

Artificial Intelligence is a dynamic field with continuous advancements and new developments. By staying informed about the latest trends and key players in the AI industry, individuals can harness the power of AI to drive innovation, solve complex problems, and create a better future. As we navigate the ethical challenges and opportunities presented by AI technologies, it is essential to advocate for responsible AI development and ensure that AI benefits society as a whole.

Stay informed, engage with the AI community, and be a part of shaping the future of Artificial Intelligence. Together, we can unlock the full potential of AI and build a more intelligent and ethical world.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).